In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from skopt.space import Real, Integer
from modules.interface import RunModel, RunOptimization

In [2]:
# --- INÍCIO DO SCRIPT DE TESTE ---

# 1. Carregar o Banco de Dados (California Housing)
print("1. Carregando o banco de dados...")
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target)

print("Formato dos dados (features):", X.shape)
print("Amostra dos dados:")
print(X.head())

# 2. Dividir os dados em Conjunto de Treino e Teste
print("\n2. Dividindo os dados em treino e teste...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Tamanho do treino: {len(X_train)} amostras")
print(f"Tamanho do teste: {len(X_test)} amostras")


# --- OTIMIZAÇÃO PARA LIGHTGBM ---
print("="*50)
print("INICIANDO OTIMIZAÇÃO PARA LIGHTGBM")
print("="*50)

space_lgbm = [
    Integer(20, 100, name='num_leaves'),
    Integer(3, 15, name='max_depth'),
    Real(0.01, 0.2, name='learning_rate', prior='log-uniform')
]

fixed_params_lgbm = {
    'objective': 'mae', 
    'metric': 'mae', 
    'num_iterations ': 1000,
    'random_state': 42, 
    'n_jobs': -1, 
    'verbose': -1
}

calibration_kwargs = {'callbacks':[lgb.early_stopping(stopping_rounds=10, verbose=False)]}
optimization_kwargs = {'n_initial_points': 15,
                        'n_calls': 30,
                        'initial_point_generator': 'random',
                        'random_state': 42,

                       
                       }

# Simplesmente passe 'model_type="lightgbm"'
optimizer_lgbm = RunOptimization().run_multiple_optimizations(opt_class='gp_minimize',
    model_class="lgbm",
    datasets=[X_train, X_test, y_train, y_test],
    space=space_lgbm, 
    fixed_params=fixed_params_lgbm,
    metric='mae',
    calibration_kwargs=calibration_kwargs,
    optimization_kwargs= optimization_kwargs

)

print(f"\nMelhor MAE para LightGBM: {optimizer_lgbm[0].fun:.4f}")
print(f"Melhores parâmetros: {optimizer_lgbm[0].x}")

1. Carregando o banco de dados...
Formato dos dados (features): (20640, 8)
Amostra dos dados:
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25  

2. Dividindo os dados em treino e teste...
Tamanho do treino: 16512 amostras
Tamanho do teste: 4128 amostras
INICIANDO OTIMIZAÇÃO PARA LIGHTGBM
[INFO] Estrutura de dados 1D detectada. Adicionando uma dimensão para compatibilidade.

[INFO] Running optimization 4
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<clas

In [3]:
print(f"\nMelhor MAE para LightGBM: {optimizer_lgbm[0].fun:.4f}")
print(f"Melhores parâmetros: {optimizer_lgbm[0].x}")


Melhor MAE para LightGBM: 0.2924
Melhores parâmetros: [np.int64(100), np.int64(15), 0.1303792958093324]


In [ ]:
# # Agora, para otimizar outro modelo, só mudamos o tipo e os parâmetros!
# optimizer_xgb = BayesianOptimization(
#     model_type="xgboost",
#     X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
#     space=space_xgb, fixed_params=fixed_params_xgb
# )
# resultado_xgb = optimizer_xgb.run(n_calls=15, random_state=42)
# print(f"\nMelhor MAE para XGBoost: {resultado_xgb.fun:.4f}")
# print(f"Melhores parâmetros: {resultado_xgb.x}")